<a href="https://colab.research.google.com/github/CryptoVizArt/kiboMoney_riskDashboard_streamlit/blob/main/kibo_sample_signalExtraction_000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd

def fetch_single_data(url, start_date="2023-01-01", end_date="2025-03-14"):
    start_year = int(start_date.split('-')[0])
    end_year = int(end_date.split('-')[0])
    years = list(range(start_year, end_year + 1))

    all_data = {}

    for year in years:
        api_url = f"{url}?chunk={year}"
        try:
            response = requests.get(api_url)
            data = response.json()
            if 'dataset' in data and 'map' in data['dataset']:
                all_data.update(data['dataset']['map'])
        except Exception:
            pass

    if all_data:
        df = pd.DataFrame({
            'date': list(all_data.keys()),
            'value': list(all_data.values())
        })
        df['date'] = pd.to_datetime(df['date'])
        df = df[(df['date'] >= pd.to_datetime(start_date)) &
                (df['date'] <= pd.to_datetime(end_date))]
        df = df.sort_values('date').reset_index(drop=True)
        return df

    return pd.DataFrame(columns=['date', 'value'])

def fetch_all_metrics(metrics_urls, start_date="2023-01-01", end_date="2025-03-14"):
    # Dictionary to hold all dataframes
    metric_dfs = {}

    for url in metrics_urls:
        metric_name = url.split('/')[-1]
        df = fetch_single_data(url, start_date, end_date)

        if not df.empty:
            # Store dataframe with metric name as key
            metric_dfs[metric_name] = df.set_index('date')['value']

    if metric_dfs:
        # Combine all series side by side with date as index
        raw_data = pd.DataFrame(metric_dfs)
        return raw_data

    return pd.DataFrame()

# Example usage:
METRICS = [
    "https://kibo.money/api/date-to-realized-cap",
    "https://kibo.money/api/date-to-supply-in-profit-to-circulating-supply-ratio",
    "https://kibo.money/api/date-to-market-cap",
    "https://kibo.money/api/date-to-realized-profit-to-loss-1d-sum-ratio"
]

start_date = "2021-01-01"
end_date = "2025-03-14"

raw_data = fetch_all_metrics(METRICS, start_date, end_date)
print(f"Retrieved data for {raw_data.shape[1]} metrics across {raw_data.shape[0]} dates")
print(raw_data.head())

Retrieved data for 4 metrics across 1534 dates
            date-to-realized-cap  \
date                               
2021-01-01          1.729444e+11   
2021-01-02          1.759768e+11   
2021-01-03          1.791336e+11   
2021-01-04          1.818415e+11   
2021-01-05          1.845234e+11   

            date-to-supply-in-profit-to-circulating-supply-ratio  \
date                                                               
2021-01-01                                          97.979227      
2021-01-02                                          98.036590      
2021-01-03                                          98.227691      
2021-01-04                                          99.095707      
2021-01-05                                          97.378188      

            date-to-market-cap  date-to-realized-profit-to-loss-1d-sum-ratio  
date                                                                          
2021-01-01        5.382895e+11                                   

In [ ]:
""